## 浙江省财政收支结构特征分析

In [18]:
import polars as pl
import altair as alt
import pandas as pd
import numpy as np

# ==============================================
# 1. 读取财政收支数据
# ==============================================
df = pl.read_excel(
    "../data/12-2 地方一般预算收入和支出（2018-2022年）.xlsx",
    sheet_name="12-2 地方一般预算收入和支出（2018-2022年）",
    read_options={"header_row": 2}
)

print("数据读取成功，列名：", df.columns)

# 关键步骤：去除指标列的空格
df = df.with_columns([
    pl.col("项目").str.strip_chars().alias("项目")  # 去除前后空格
])

数据读取成功，列名： ['项目', '2018', '2019', '2020', '2021', '2022', '2023']


In [20]:
# ==============================================
# 2. 财政收入结构分析
# ==============================================
# 提取相关收入数据行
income_items = [
    "一般预算收入合计",
    "税收收入小计", 
    "非税收入小计",
    "国内增值税",
    "企业所得税",
    "个人所得税",
    "城市维护建设税",
    "房产税",
    "契税",
    "其它地方各税"
]

years = ["2018", "2019", "2020", "2021", "2022", "2023"]

# 创建收入结构数据
income_data = {}
for item in income_items:
    row = df.filter(pl.col("项目") == item)
    if len(row) > 0:
        values = [float(row[year][0]) for year in years]
        income_data[item] = values

#输出收入结构数据预览
df_income = pd.DataFrame(income_data, index=years)
print("\n收入结构数据预览：", df_income)


收入结构数据预览：       一般预算收入合计   税收收入小计   非税收入小计    国内增值税    企业所得税   个人所得税  城市维护建设税     房产税  \
2018   6598.08  5586.63  1011.58  2408.84   972.46  465.86   370.55  233.29   
2019   7048.00  5898.17  1149.83  2374.58  1115.55  412.94   363.26  207.25   
2020   7248.24  6261.75   986.49  2317.15  1145.02  468.79   356.74  234.12   
2021   8262.57  7171.91  1090.66  2687.98  1443.97  532.11   413.74  249.82   
2022   8039.38  6619.86  1419.53  2251.36  1517.29  592.16   416.12  322.36   
2023   8600.02  7124.06  1475.96  3011.46  1311.36  582.55   405.02  348.54   

          契税   其它地方各税  
2018  459.45   681.36  
2019  571.34   853.29  
2020  838.16   901.76  
2021  767.28  1077.01  
2022  605.30   915.27  
2023  609.13   856.00  


In [21]:

# 创建收入结构DataFrame
income_df = pl.DataFrame({
    "年份": years
})

for item, values in income_data.items():
    income_df = income_df.with_columns([
        pl.Series(item, values)
    ])

print("\n财政收入结构数据：")
print(income_df)


财政收入结构数据：
shape: (6, 11)
┌──────┬─────────────┬─────────────┬─────────────┬───┬─────────────┬────────┬────────┬─────────────┐
│ 年份 ┆ 一般预算收  ┆ 税收收入小  ┆ 非税收入小  ┆ … ┆ 城市维护建  ┆ 房产税 ┆ 契税   ┆ 其它地方各  │
│ ---  ┆ 入合计      ┆ 计          ┆ 计          ┆   ┆ 设税        ┆ ---    ┆ ---    ┆ 税          │
│ str  ┆ ---         ┆ ---         ┆ ---         ┆   ┆ ---         ┆ f64    ┆ f64    ┆ ---         │
│      ┆ f64         ┆ f64         ┆ f64         ┆   ┆ f64         ┆        ┆        ┆ f64         │
╞══════╪═════════════╪═════════════╪═════════════╪═══╪═════════════╪════════╪════════╪═════════════╡
│ 2018 ┆ 6598.08     ┆ 5586.63     ┆ 1011.58     ┆ … ┆ 370.55      ┆ 233.29 ┆ 459.45 ┆ 681.36      │
│ 2019 ┆ 7048.0      ┆ 5898.17     ┆ 1149.83     ┆ … ┆ 363.26      ┆ 207.25 ┆ 571.34 ┆ 853.29      │
│ 2020 ┆ 7248.24     ┆ 6261.75     ┆ 986.49      ┆ … ┆ 356.74      ┆ 234.12 ┆ 838.16 ┆ 901.76      │
│ 2021 ┆ 8262.57     ┆ 7171.91     ┆ 1090.66     ┆ … ┆ 413.74      ┆ 249.82 ┆ 767.28 ┆ 1077.01     │
│ 20

In [22]:
# 计算税收占比
income_df = income_df.with_columns([
    (pl.col("税收收入小计") / pl.col("一般预算收入合计") * 100).alias("税收收入占比(%)"),
    (pl.col("非税收入小计") / pl.col("一般预算收入合计") * 100).alias("非税收入占比(%)")
])

# 计算主要税种占比（占税收收入）
income_df = income_df.with_columns([
    (pl.col("国内增值税") / pl.col("税收收入小计") * 100).alias("增值税占比(%)"),
    (pl.col("企业所得税") / pl.col("税收收入小计") * 100).alias("企业所得税占比(%)"),
    (pl.col("个人所得税") / pl.col("税收收入小计") * 100).alias("个人所得税占比(%)")
])

# 计算三大税种合计占比
income_df = income_df.with_columns([
    ((pl.col("国内增值税") + pl.col("企业所得税") + pl.col("个人所得税")) / 
     pl.col("税收收入小计") * 100).alias("三大税种合计占比(%)")
])

print("\n财政收入结构分析：")
print(income_df.select([
    "年份", "一般预算收入合计", "税收收入小计", "税收收入占比(%)",
    "增值税占比(%)", "企业所得税占比(%)", "个人所得税占比(%)",
    "三大税种合计占比(%)"
]).to_pandas().to_string(index=False))


财政收入结构分析：
  年份  一般预算收入合计  税收收入小计  税收收入占比(%)  增值税占比(%)  企业所得税占比(%)  个人所得税占比(%)  三大税种合计占比(%)
2018   6598.08 5586.63  84.670541 43.117944   17.406916    8.338838    68.863698
2019   7048.00 5898.17  83.685726 40.259606   18.913494    7.001155    66.174254
2020   7248.24 6261.75  86.389937 37.004831   18.285942    7.486565    62.777339
2021   8262.57 7171.91  86.799991 37.479277   20.133688    7.419362    65.032328
2022   8039.38 6619.86  82.342917 34.009178   22.920273    8.945204    65.874656
2023   8600.02 7124.06  82.837714 42.271682   18.407481    8.177219    68.856382


In [23]:
# ==============================================
# 3. 财政支出结构分析
# ==============================================
# 提取相关支出数据行
expense_items = [
    "一般预算支出合计",
    "一般公共服务",
    "公共安全",
    "教育",
    "科学技术", 
    "文化旅游体育与传媒",
    "社会保障和就业",
    "卫生健康",
    "节能环保",
    "城乡社区",
    "农林水",
    "交通运输",
    "工业商业等事务",
    "其他支出"
]

# 创建支出结构数据
expense_data = {}
for item in expense_items:
    row = df.filter(pl.col("项目") == item)
    if len(row) > 0:
        values = [float(row[year][0]) for year in years]
        expense_data[item] = values
        
print("\n支出结构数据预览：", pd.DataFrame(expense_data, index=years))


支出结构数据预览：       一般预算支出合计   一般公共服务    公共安全       教育    科学技术  文化旅游体育与传媒  社会保障和就业     卫生健康  \
2018   8627.51   875.33  614.24  1572.47  379.66     174.59   914.93   626.20   
2019  10053.03  1009.96  662.70  1764.69  516.06     203.26  1073.94   735.61   
2020  10082.01  1051.55  691.96  1881.09  472.13     229.62  1130.02   838.85   
2021  11016.87  1105.34  740.38  2039.98  578.58     249.35  1286.13   908.04   
2022  12017.70  1162.01  749.75  2199.26  680.89     269.28  1400.55  1236.08   
2023  12353.09  1169.86  763.08  2365.17  787.48     291.44  1497.70  1148.02   

        节能环保     城乡社区     农林水    交通运输  工业商业等事务    其他支出  
2018  194.75  1158.88  724.46  391.29   549.79  451.64  
2019  269.55  1627.80  744.24  444.59   470.11  530.48  
2020  220.59  1021.67  764.89  470.50   714.52  594.62  
2021  203.78  1137.51  795.27  514.32   834.81  623.38  
2022  223.16  1099.25  831.20  589.37   893.35  670.83  
2023  187.98  1094.06  910.65  547.19   919.22  656.68  


In [24]:
# 创建支出结构DataFrame
expense_df = pl.DataFrame({
    "年份": years
})

for item, values in expense_data.items():
    expense_df = expense_df.with_columns([
        pl.Series(item, values)
    ])

print("\n财政支出结构数据：")
print(expense_df)


财政支出结构数据：
shape: (6, 15)
┌──────┬───────────────┬──────────────┬──────────┬───┬────────┬──────────┬──────────────┬──────────┐
│ 年份 ┆ 一般预算支出  ┆ 一般公共服务 ┆ 公共安全 ┆ … ┆ 农林水 ┆ 交通运输 ┆ 工业商业等事 ┆ 其他支出 │
│ ---  ┆ 合计          ┆ ---          ┆ ---      ┆   ┆ ---    ┆ ---      ┆ 务           ┆ ---      │
│ str  ┆ ---           ┆ f64          ┆ f64      ┆   ┆ f64    ┆ f64      ┆ ---          ┆ f64      │
│      ┆ f64           ┆              ┆          ┆   ┆        ┆          ┆ f64          ┆          │
╞══════╪═══════════════╪══════════════╪══════════╪═══╪════════╪══════════╪══════════════╪══════════╡
│ 2018 ┆ 8627.51       ┆ 875.33       ┆ 614.24   ┆ … ┆ 724.46 ┆ 391.29   ┆ 549.79       ┆ 451.64   │
│ 2019 ┆ 10053.03      ┆ 1009.96      ┆ 662.7    ┆ … ┆ 744.24 ┆ 444.59   ┆ 470.11       ┆ 530.48   │
│ 2020 ┆ 10082.01      ┆ 1051.55      ┆ 691.96   ┆ … ┆ 764.89 ┆ 470.5    ┆ 714.52       ┆ 594.62   │
│ 2021 ┆ 11016.87      ┆ 1105.34      ┆ 740.38   ┆ … ┆ 795.27 ┆ 514.32   ┆ 834.81       ┆ 623.38   │
│ 

In [25]:
# 计算各类支出占比
expense_df = expense_df.with_columns([
    (pl.col("教育") / pl.col("一般预算支出合计") * 100).alias("教育支出占比(%)"),
    (pl.col("科学技术") / pl.col("一般预算支出合计") * 100).alias("科技支出占比(%)"),
    (pl.col("社会保障和就业") / pl.col("一般预算支出合计") * 100).alias("社保就业支出占比(%)"),
    (pl.col("卫生健康") / pl.col("一般预算支出合计") * 100).alias("卫生健康支出占比(%)"),
    (pl.col("节能环保") / pl.col("一般预算支出合计") * 100).alias("节能环保支出占比(%)"),
    (pl.col("城乡社区") / pl.col("一般预算支出合计") * 100).alias("城乡社区支出占比(%)")
])

# 计算民生支出合计（教育、社保就业、卫生健康）
expense_df = expense_df.with_columns([
    (pl.col("教育") + pl.col("社会保障和就业") + pl.col("卫生健康")).alias("民生支出合计"),
    ((pl.col("教育") + pl.col("社会保障和就业") + pl.col("卫生健康")) / 
     pl.col("一般预算支出合计") * 100).alias("民生支出占比(%)")
])

# 计算发展类支出（科技、城乡社区、交通运输、工业商业等事务）
expense_df = expense_df.with_columns([
    (pl.col("科学技术") + pl.col("城乡社区") + pl.col("交通运输") + pl.col("工业商业等事务")).alias("发展支出合计"),
    ((pl.col("科学技术") + pl.col("城乡社区") + pl.col("交通运输") + pl.col("工业商业等事务")) / 
     pl.col("一般预算支出合计") * 100).alias("发展支出占比(%)")
])

print("\n财政支出结构分析：")
print(expense_df.select([
    "年份", "一般预算支出合计", "民生支出合计", "民生支出占比(%)",
    "教育支出占比(%)", "社保就业支出占比(%)", "卫生健康支出占比(%)",
    "科技支出占比(%)", "发展支出占比(%)"
]).to_pandas().to_string(index=False))


财政支出结构分析：
  年份  一般预算支出合计  民生支出合计  民生支出占比(%)  教育支出占比(%)  社保就业支出占比(%)  卫生健康支出占比(%)  科技支出占比(%)  发展支出占比(%)
2018   8627.51 3113.60  36.089208  18.226232    10.604798     7.258178   4.400574  28.740853
2019  10053.03 3574.24  35.553858  17.553812    10.682749     7.317296   5.133378  30.424260
2020  10082.01 3849.96  38.186433  18.657887    11.208281     8.320266   4.682896  26.570297
2021  11016.87 4234.15  38.433330  18.516875    11.674187     8.242268   5.251764  27.822966
2022  12017.70 4835.89  40.239730  18.300174    11.654060    10.285496   5.665726  27.150453
2023  12353.09 5010.89  40.563859  19.146384    12.124092     9.293383   6.374761  27.102126


In [26]:
# ==============================================
# 4. 计算增长情况
# ==============================================
# 计算累计增长
for df_type, df_name in [(income_df, "收入"), (expense_df, "支出")]:
    print(f"\n{df_name}累计增长情况：")
    for col in ["税收收入小计", "国内增值税", "企业所得税", "个人所得税"]:
        if col in df_type.columns:
            start = df_type[col][0]
            end = df_type[col][-1]
            growth = (end - start) / start * 100
            print(f"  {col}: {growth:.1f}%")


收入累计增长情况：
  税收收入小计: 27.5%
  国内增值税: 25.0%
  企业所得税: 34.8%
  个人所得税: 25.0%

支出累计增长情况：


In [28]:
# ==============================================
# 5. 数据可视化
# ==============================================
# 转换为Pandas用于Altair
income_pd = income_df.to_pandas()
expense_pd = expense_df.to_pandas()

# A. 财政收入结构可视化
# 1. 税收与非税收入占比趋势
tax_structure = pd.melt(
    income_pd[['年份', '税收收入占比(%)', '非税收入占比(%)']],
    id_vars=['年份'],
    value_vars=['税收收入占比(%)', '非税收入占比(%)'],
    var_name='收入类型',
    value_name='占比(%)'
)

chart1 = alt.Chart(tax_structure).mark_area(
    opacity=0.6,
    line=True
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('占比(%):Q', axis=alt.Axis(title='占比(%)'), stack="normalize"),
    color=alt.Color('收入类型:N',
                   scale=alt.Scale(
                       domain=['税收收入占比(%)', '非税收入占比(%)'],
                       range=['#4C78A8', '#F58518']
                   ),
                   legend=alt.Legend(title="收入类型"))
).properties(
    title='浙江省税收与非税收入占比趋势（2018-2023年）',
    width=600,
    height=400
)
chart1.show()

alt.Chart(...)

In [29]:
# 2. 主要税种占比堆叠图
tax_composition = pd.melt(
    income_pd[['年份', '增值税占比(%)', '企业所得税占比(%)', '个人所得税占比(%)']],
    id_vars=['年份'],
    value_vars=['增值税占比(%)', '企业所得税占比(%)', '个人所得税占比(%)'],
    var_name='税种',
    value_name='占比(%)'
)

chart2 = alt.Chart(tax_composition).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('占比(%):Q', axis=alt.Axis(title='占税收收入比重(%)')),
    color=alt.Color('税种:N',
                   scale=alt.Scale(
                       domain=['增值税占比(%)', '企业所得税占比(%)', '个人所得税占比(%)'],
                       range=['#1F77B4', '#FF7F0E', '#2CA02C']
                   ),
                   legend=alt.Legend(title="主要税种")),
    tooltip=['年份', '税种', alt.Tooltip('占比(%):Q', format='.1f')]
).properties(
    title='浙江省主要税种占税收收入比重',
    width=600,
    height=400
)
chart2.show()

alt.Chart(...)

In [31]:
# 3. 三大税种合计占比趋势
chart3 = alt.Chart(income_pd).mark_line(
    point=True,
    color='#E45756',
    strokeWidth=3
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('三大税种合计占比(%):Q', axis=alt.Axis(title='三大税种合计占比(%)')),
    tooltip=['年份', alt.Tooltip('三大税种合计占比(%):Q', format='.1f')]
).properties(
    title='浙江省增值税、企业所得税、个人所得税合计占比趋势',
    width=600,
    height=300
)

# 添加目标线（68%）
target_line = alt.Chart(pd.DataFrame({'y': [68]})).mark_rule(
    strokeDash=[5, 5],
    color='gray',
    opacity=0.7
).encode(y='y:Q')

chart3 = chart3 + target_line
chart3.show()

alt.LayerChart(...)

In [32]:

# B. 财政支出结构可视化
# 4. 民生支出占比趋势
chart4 = alt.Chart(expense_pd).mark_line(
    point=True,
    color='#9467BD',
    strokeWidth=3
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('民生支出占比(%):Q', axis=alt.Axis(title='民生支出占比(%)')),
    tooltip=['年份', alt.Tooltip('民生支出占比(%):Q', format='.1f')]
).properties(
    title='浙江省民生支出占比趋势（教育、社保就业、卫生健康）',
    width=600,
    height=300
)
chart4.show()

alt.Chart(...)

In [36]:
# 5. 主要支出项目占比堆叠图
expense_composition = pd.melt(
    expense_pd[['年份', '教育支出占比(%)', '社保就业支出占比(%)', '卫生健康支出占比(%)', 
                '科技支出占比(%)', '节能环保支出占比(%)']],
    id_vars=['年份'],
    value_vars=['教育支出占比(%)', '社保就业支出占比(%)', '卫生健康支出占比(%)', 
                '科技支出占比(%)', '节能环保支出占比(%)'],
    var_name='支出项目',
    value_name='占比(%)'
)

chart5 = alt.Chart(expense_composition).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('占比(%):Q', axis=alt.Axis(title='占财政支出比重(%)')),
    color=alt.Color('支出项目:N',
                   scale=alt.Scale(
                       domain=['教育支出占比(%)', '社保就业支出占比(%)', '卫生健康支出占比(%)',
                               '科技支出占比(%)', '节能环保支出占比(%)'],
                       range=['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD']
                   ),
                   legend=alt.Legend(title="支出项目", columns=1)),
    tooltip=['年份', '支出项目', alt.Tooltip('占比(%):Q', format='.1f')]
).properties(
    title='浙江省主要财政支出项目占比',
    width=600,
    height=400
)
chart5.show()

alt.Chart(...)

In [34]:
# 6. 民生与发展支出对比
development_comparison = pd.melt(
    expense_pd[['年份', '民生支出占比(%)', '发展支出占比(%)']],
    id_vars=['年份'],
    value_vars=['民生支出占比(%)', '发展支出占比(%)'],
    var_name='支出类型',
    value_name='占比(%)'
)

chart6 = alt.Chart(development_comparison).mark_area(
    opacity=0.5,
    line=True
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('占比(%):Q', axis=alt.Axis(title='占比(%)')),
    color=alt.Color('支出类型:N',
                   scale=alt.Scale(
                       domain=['民生支出占比(%)', '发展支出占比(%)'],
                       range=['#4C78A8', '#F58518']
                   ),
                   legend=alt.Legend(title="支出类型"))
).properties(
    title='浙江省民生与发展支出占比对比',
    width=600,
    height=400
)
chart6.show()

alt.Chart(...)

In [37]:

# 7. 各年度支出结构桑基图（可选）
# 准备桑基图数据
expense_categories = ['教育', '科学技术', '社会保障和就业', '卫生健康', 
                     '节能环保', '城乡社区', '交通运输', '其他支出']

sankey_data = []
for year in years:
    total = expense_pd[expense_pd['年份'] == year]['一般预算支出合计'].values[0]
    for category in expense_categories:
        if category in expense_pd.columns:
            value = expense_pd[expense_pd['年份'] == year][category].values[0]
            percentage = (value / total * 100)
            sankey_data.append({
                '年份': year,
                '支出类别': category,
                '金额': value,
                '占比': percentage
            })

sankey_df = pd.DataFrame(sankey_data)

# 创建桑基图风格的堆叠条形图
chart7 = alt.Chart(sankey_df).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('金额:Q', axis=alt.Axis(title='支出金额（亿元）'), stack="normalize"),
    color=alt.Color('支出类别:N',
                   scale=alt.Scale(
                       domain=expense_categories,
                       range=['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728',
                              '#9467BD', '#8C564B', '#E377C2', '#7F7F7F']
                   ),
                   legend=alt.Legend(title="支出类别", columns=1)),
    tooltip=['年份', '支出类别', alt.Tooltip('金额:Q', format=',.0f'), 
             alt.Tooltip('占比:Q', format='.1f')]
).properties(
    title='浙江省财政支出结构变化（2018-2023年）',
    width=700,
    height=500
)

chart7.show()

alt.Chart(...)